In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import scipy
import numpy as np

In [10]:
task = 'Indoor_Walk'


participants = {'1': {1:[], 2:[], 5: [], 6:[]},'2': {1:[], 2:[], 5: [], 6:[]},'6': {5:[], 6:[]}}


path = './data/Extracted_Data/%s/Labels/'%task
for filename in os.listdir(path):
    participant = filename.split('_')[1]
    labeler = int(filename[-5])
    if participant in participants.keys():
        mat = scipy.io.loadmat('./data/Extracted_Data/%s/Labels/%s'%(task, filename))
        label = []
        label = np.array(mat['LabelData']['Labels'][0])[0][:,0]
        label = label.tolist()
        participants[participant][labeler] = label


In [11]:
participant = '2'
majority = []
vote_tresholds = [1,2,3,4]
give_zero_lower_prios = [0, 1, 2, 3]

for vote_treshold in vote_tresholds:
    for give_zero_lower_prio in give_zero_lower_prios:
        for index in range(0,len(participants[participant][1])):
            vote_list = []
            maybe_fuckup = False
            numbers = {0:0, 1:0, 2:0, 3:0, 4:0, 5:0}
            vote_list = [participants[participant][1][index], participants[participant][2][index], participants[participant][5][index], participants[participant][6][index]]
            for number in numbers.keys():
                numbers[number] = vote_list.count(number)
            
            #Experimental, since 0 is no information, its votes could be lowered
            numbers[0] = max(0, numbers[0] - give_zero_lower_prio)

            if len([k for k, v in numbers.items() if v == max(numbers.values())]) > 1:
                majority.append(max([k for k, v in numbers.items() if v == max(numbers.values())]))     #Max could be changed to min or entire append stuff to 0

            else:
                if max(numbers.values()) >= vote_treshold:
                    majority.append(max(numbers, key=numbers.get))

                else:
                    majority.append(0)

            prev_choice = majority[-1]

        with open('./generated_data/majority_vote/%s/max label/participant_%s_vt_%s_zlp_%s.txt'%(task, participant, vote_treshold ,give_zero_lower_prio), 'w') as f:
            f.write(str(majority))

In [12]:
len(majority)

1591680